In [ ]:
import speech_recognition as sr
import re
import pyttsx3
from datetime import datetime
engine = pyttsx3.init()

import sqlite3

# Connect to SQLite database (it will create the database if it doesn't exist)
conn = sqlite3.connect('task_management.db')

# Create a cursor object to interact with the database
cur = conn.cursor()

# Create a table for storing tasks
cur.execute('''
CREATE TABLE IF NOT EXISTS tasks (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    task_name TEXT NOT NULL,
    due_date TEXT,
    status TEXT DEFAULT 'Pending',
    priority TEXT
)
''')

# Commit the changes
conn.commit()

# Close the connection (we will reconnect as needed later)
conn.close()


def add_task(task):
    # Get user input for task details
    due_date = input("Enter due date (YYYY-MM-DD) [Leave empty if none]: ")
    priority = input("Enter task priority (Low/Medium/High): ")
    if due_date and not validate_date(due_date):
        print("Invalid date format. Please enter in YYYY-MM-DD format.")
        return

    # Connect to the database
    conn = sqlite3.connect('task_management.db')
    cur = conn.cursor()

    # Insert the task into the database
    cur.execute("INSERT INTO tasks (task_name, due_date, priority) VALUES (?, ?, ?)",
                (task, due_date if due_date else None, priority))

    # Commit the changes and close the connection
    conn.commit()
    conn.close()

    print("Task added successfully!")


def view_tasks():
    # Connect to the database
    conn = sqlite3.connect('task_management.db')
    cur = conn.cursor()

    # Fetch all tasks from the database
    cur.execute("SELECT id, task_name, due_date, status, priority FROM tasks")
    tasks = cur.fetchall()

    # Display the tasks in a table format
    if tasks:
        # Print header row
        print(f"{'ID':<5} {'Task Name':<20} {'Due Date':<15} {'Status':<10} {'Priority':<10}")
        print('-' * 60)  # separator line

        # Print task rows
        for task in tasks:
            print(f"{task[0]:<5} {task[1]:<20} {task[2]:<15} {task[3]:<10} {task[4]:<10}")
        print("\n\n\n")
    else:
        print("No tasks found.")

    # Close the connection
    conn.close()


def delete_task(task_number):
    

    # Connect to the database
    conn = sqlite3.connect('task_management.db')
    cur = conn.cursor()

    # Delete the task from the database
    cur.execute("DELETE FROM tasks WHERE id = ?", (task_number,))

    # Commit the changes and close the connection
    conn.commit()
    conn.close()

    print(f"Task {task_number} deleted successfully!")





patterns = {
    "add task": r'(add)[a-zA-Z\s]*(task|work)',  # add task
    "view task": r'(view|see|show|pending|display)',  # view task
    "delete task": r'(delete|remove|completed|done)',  # delete task
    "update task status":r'(update|change|modify)[a-zA-Z\s]*(status|state)' #update task
    
}
def speak(text):
    engine.say(text)
    engine.runAndWait()

def listen():
    recognizer = sr.Recognizer()
    mic = sr.Microphone()

    with mic as source:
        print("Listening for a command...")
        audio = recognizer.listen(source)

    try:
        command = recognizer.recognize_google(audio)
        print("You said:",command)
        return command.lower()
    except sr.UnknownValueError:
        speak("Sorry, I didn't catch that.")
        print("Sorry, I didn't catch that.")
        return None
    except sr.RequestError as e:
        speak("Sorry, I'm having trouble connecting to the speech service.")
        print("Sorry, I'm having trouble connecting to the speech service;{0}".format(e))
        return None
    


def speech_input_interface():
    print("listening....")
    txt=listen()
    command=check_matching_patterns(txt, patterns)
    for x in command:
                if(x=="add task"):
                    print("what is the task, you want to add?")
                    speak("what is the task, you want to add?")
                    task=listen()
                    add_task(task)
                elif(x=="view task"):
                    view_tasks()
                elif(x=="delete task"):
                    print("please say the task number you want to delete")
                    speak("please say the task number you want to delete")
                    task=listen()
                    delete_task(int(task))
                elif action == "update task status":
                    speak("Please say the task number followed by the new status (e.g., task 2 in progress).")
                    task_status = listen()
                    match = re.match(r"task (\d+) (.*)", task_status)
                    if match:
                        task_number = int(match.group(1))
                        status = match.group(2).capitalize()
                        update_task_status(task_number, status)
                    else:
                        speak("Sorry, I couldn't understand the task number or status.")
                else:
                    speak("unvalid command")
                    print("unvalid command")

def text_input_interface():
    while True:  # Start a loop to keep showing the menu until the user chooses to exit
        print("1. Add Task")
        print("2. View Tasks")
        print("3. Remove Task")
        print("4. Exit")

        choice = input("Choose an option (1-4): ")
        
        if choice == '1':
            task = input("Enter the task description: ")
            add_task(task)  # Assuming add_task is defined elsewhere
        elif choice == '2':
            view_tasks()  # Assuming view_tasks is defined elsewhere
        elif choice == '3':
            task_number = int(input("Enter the task number to delete: "))
            delete_task(task_number)  # Assuming delete_task is defined elsewhere
        elif choice == '4':
            print("Exiting...")
            speak("Goodbye!")  # Assuming speak is defined elsewhere
            return  # Now, break will exit the loop when the user chooses option 4
        else:
            print("Invalid option. Please try again.")
            speak("Invalid option. Please try again.")
    


def check_matching_patterns(txt, patterns):
    matches = []
    for pattern_name, pattern in patterns.items():
        if re.search(pattern, txt):
            matches.append(pattern_name)
    
    if matches:
        return matches
    else:
        return "No match found"

command_pattern = {
    "communicate directly": r'[a-zA-Z\s]*(directly|speech|direct)[a-zA-Z\s]*',  # speech
    "manually": r'[a-zA-Z\s]*(manually|text)[a-zA-Z\s]*',  # test

}
# matches = check_matching_patterns(txt, patterns)
def main():
    #task_manager = TaskManager()
    while True:
        speak(" Welcome to your personal task manager.")
        speak("Do you want to communicate directly with me or you want to assist me manauly?")
        print("say \n communicate directly : if you want to comunicate through speech...\n manually: if you want to enter task details manually")
        command=listen()
        x=check_matching_patterns(command,command_pattern)
        if(x=="communicate directly"):   #speech wale me jana h
            speak("how can I assist you? Please speak")
            speech_input_interface()
        if(x=="manually"):                #text wale me  jana h
            text_input_interface()
            break
                
        if command is None:
            continue


if __name__ == "__main__":
    main()
